## Access
<table class="tfo-notebook-buttons" align="left">

  <td>
    <a target="_blank" href="https://colab.research.google.com/drive/1osMp_Nve0o0TI6OM2f3CIAMxjt2peQ6g?usp=sharing"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/wjlgatech/MentalCareApp/blob/master/MentalCare_by_12xFuture.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>


# Design Docs
- [WordAlive -- Reprograming Your Words to Make You fully Alive
](https://docs.google.com/document/d/1b85aPSSubMDez0wAIUzGHNLGWAx_KFLe5au9b01Lht0/edit?usp=sharing)

- [Hackathon 12XFuture prompts for Depression](https://docs.google.com/document/d/1U_Pp1GFNha3zriA4_xrt6G_nZuGFoQQdYlXy6iIqt10/edit?usp=sharing)

# Setup

In [2]:
!pip install openai
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.3/14.3 MB 47.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 58.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 KB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.9/56.9 KB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.7/140.7 KB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 KB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 KB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 KB 11.5 MB/s eta 0:00:00

In [1]:
# user info (collected through user profile)
name = "Rose"
gender = "Female"
age = "30 to 40"
profession = "software developer"
marriage = "single mom"
struggle_ls = ["depression", "unemployment", "anxiety"]

# MentalCare: ChatGPT + DALLE + Gradio



In [ ]:
import gradio as gr
import os
import openai
import requests
import json

api_key = "YOUR-API-KEY"
openai.api_key = api_key #os.environ.get("OPENAI_API_KEY")

context_prompt = f"""The following is a conversation between 'Future-You', an AI assistant, and a real person {name}. 
{name} is a {gender} of {age} years old; {name}'s profession is a {profession}; {name} is {marriage}; {name} is recently struggling with {','.join(struggle_ls)}.
The AI assistant 'Future-You' is helpful, creative, clever, and very friendly.\n\nFuture-You: I am the Future-You, a conversational AI engine designed and nurtured by You. 
After 'Future-You' answers the question from the user 'You', following by asking the user one and only one question from one of the following options. 
Options include:
          - Sympathy: Please show a deep understanding of the situation, show deep compassion to comfort and to care for the struggling soul.
          - Wording: Please provide 3 to 5 simple, easily visualizable sentences you can speak to yourself, even with a great sense of humor.
          - Action:  Please suggest physical exercise, journaling, and volunteering work that shifts the struggling person's perspective, and increases strength, skills, stemma, and symphony.
          - Visualization: Please create a mental picture and inner movie in vivid detail of what the wonderful and happy results could look like.
          - Emotion: Please recommend song and pure music to comfort, encourage, and inspire the struggling person.',
          - Sharing: Please write a blog post about the person overcoming experience and share it with family and friends of the user's choice. Ask the person if there is someone he/she can help.
The answer 'Future-You' provide should always include these 4 parts: 
1) showing sympathy or affirmation, 
2) providing suggestions, following the Options mentioned above
3) providing an explanation on why that suggestions work
4) asking the user new question, in order to helping the user to understand the deeper root cause of problem OR strengthen the positive momentum the user has made

The language 'Future-You' uses should be easily visualizable, actionable, compassionate and very humorous!
"""

# generate image by DALLE
import requests
def generate_image(prompt):
    url = "https://api.openai.com/v1/images/generations"
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {api_key}"
    }
    data = {
        "model": "image-alpha-001",
        "prompt": prompt,
        "num_images": 1,
        "size": "512x512",
        "response_format": "url"
    }
    response = requests.post(url, headers=headers, json=data)
    response.raise_for_status()
    return response.json()["data"][0]["url"]

# generate a text using the ChatGPT API
def generate_text(prompt):
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        temperature=0.7,
        max_tokens=1024,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0
    )
    message = response.choices[0].text.strip()
    return message

# define a chatbot that either generate image OR text according to input_text
def chatbot(input_text):
    if "imag" in input_text.lower() or "visualiz" in input_text.lower() or "see" in input_text.lower():
        return generate_image(input_text), "Here's the generated image:"
    else:
        return generate_text(input_text), ""

#prompt_templates = {"Default ChatGPT": ""}
prompt_templates = {"Future You": context_prompt}

def get_empty_state():
    return {"total_tokens": 0, "messages": []}

def download_prompt_templates():
    #if url == "":
    # credits: https://github.com/f/awesome-chatgpt-prompts
    url = "https://raw.githubusercontent.com/f/awesome-chatgpt-prompts/main/prompts.csv" #"https://raw.githubusercontent.com/wjlgatech/WordsAliveApp/master/data/prompts.csv"
    response = requests.get(url)

    for line in response.text.splitlines()[1:]:
        act, prompt = line.split('","')
        prompt_templates[act.replace('"', '')] = prompt.replace('"', '')

    choices = list(prompt_templates.keys())
    return gr.update(value=choices[0], choices=choices)

def on_token_change(user_token):
    openai.api_key = user_token or os.environ.get("OPENAI_API_KEY")

def on_prompt_template_change(prompt_template):
    if not isinstance(prompt_template, str): return
    return prompt_templates[prompt_template]

def submit_message(user_token, prompt, prompt_template, temperature, max_tokens, state):

    history = state['messages']

    if not prompt:
        return gr.update(value='', visible=state['total_tokens'] < 1_000), [(history[i]['content'], history[i+1]['content']) for i in range(0, len(history)-1, 2)], f"Total tokens used: {state['total_tokens']} / 3000", state
    
    prompt_template = prompt_templates[prompt_template]

    system_prompt = []
    if prompt_template:
        system_prompt = [{ "role": "system", "content": prompt_template }]

    prompt_msg = { "role": "user", "content": prompt }
    
    try:
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=system_prompt + history + [prompt_msg], temperature=temperature, max_tokens=max_tokens)

        history.append(prompt_msg)
        history.append(completion.choices[0].message.to_dict())

        state['total_tokens'] += completion['usage']['total_tokens']
    
    except Exception as e:
        history.append(prompt_msg)
        history.append({
            "role": "system",
            "content": f"Error: {e}"
        })

    total_tokens_used_msg = f"Total tokens used: {state['total_tokens']} / 3000" if not user_token else ""
    chat_messages = [(history[i]['content'], history[i+1]['content']) for i in range(0, len(history)-1, 2)]
    input_visibility = user_token or state['total_tokens'] < 3000

    return gr.update(value='', visible=input_visibility), chat_messages, total_tokens_used_msg, state

def clear_conversation():
    return gr.update(value=None, visible=True), None, "", get_empty_state()

css = """
      #col-container {max-width: 80%; margin-left: auto; margin-right: auto;}
      #chatbox {min-height: 400px;}
      #header {text-align: center;}
      #prompt_template_preview {padding: 1em; border-width: 1px; border-style: solid; border-color: #e0e0e0; border-radius: 4px;}
      #total_tokens_str {text-align: right; font-size: 0.8em; color: #666; height: 1em;}
      #label {font-size: 0.8em; padding: 0.5em; margin: 0;}
      """

### chatbot_interface1
chatbot_interface1 = gr.Interface(
    fn=chatbot,
    inputs=gr.inputs.Textbox(label="Input"),
    outputs=[
        gr.outputs.Image(label="Generated Image", type='pil'),
        gr.outputs.Textbox(label="Output")
    ],
    title="Generating Mental Image",
    description="",
    examples=[["Hello", "Hi!"], ["Can you show me the visualization?", "Here's the generated image:"]]
)

chatbot_interface1.launch()

### chatbot_interface2


with gr.Blocks(css=css) as demo:
    
    state = gr.State(get_empty_state())


    with gr.Column(elem_id="col-container"):
        gr.Markdown("""## Mental Care
                    AI Psychological Assistant<br>
                    """,
                    elem_id="header")

        with gr.Row():
            with gr.Column():
                chatbot = gr.Chatbot(elem_id="chatbox")
                input_message = gr.Textbox(show_label=False, placeholder="Enter text and press enter", visible=True).style(container=False)
                total_tokens_str = gr.Markdown(elem_id="total_tokens_str")
                btn_clear_conversation = gr.Button("🔃 Start New Conversation")
            with gr.Column():
                prompt_template = gr.Dropdown(label="Select a template for the chatbot:", choices=list(prompt_templates.keys()))
                prompt_template_preview = gr.Markdown(elem_id="prompt_template_preview")
                gr.Markdown("Enter your own OpenAI API Key to remove the 3000 token limit. You can get it [here](https://platform.openai.com/account/api-keys).", elem_id="label")
                user_token = gr.Textbox(placeholder="OpenAI API Key", type="password", show_label=False)
                with gr.Accordion("Advanced parameters", open=False):
                    temperature = gr.Slider(minimum=0, maximum=2.0, value=0.7, step=0.1, interactive=True, label="Temperature (higher = more creative/chaotic)")
                    max_tokens = gr.Slider(minimum=100, maximum=4096, value=1000, step=1, interactive=True, label="Max tokens per response")

    gr.HTML('''<br><br><br><center><a href="https://huggingface.co/spaces/anzorq/chatgpt-demo?duplicate=true"><img src="https://bit.ly/3gLdBN6" alt="Duplicate Space"></a>You can duplicate this Space.<br>
            Don't forget to set your own <a href="https://platform.openai.com/account/api-keys">OpenAI API Key</a> environment variable in Settings.</center>''')

    input_message.submit(submit_message, [user_token, input_message, prompt_template, temperature, max_tokens, state], [input_message, chatbot, total_tokens_str, state])

    btn_clear_conversation.click(clear_conversation, [], [input_message, chatbot, total_tokens_str, state])
    prompt_template.change(on_prompt_template_change, inputs=[prompt_template], outputs=[prompt_template_preview])
    user_token.change(on_token_change, inputs=[user_token], outputs=[])

    
    demo.load(download_prompt_templates, inputs=None, outputs=[prompt_template])


demo.launch(debug=True, height='800px')



/usr/local/lib/python3.8/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:43: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/gradio/outputs.py:22: UserWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>